In [0]:
import os, shutil
shutil.rmtree('/content/HMP_Dataset')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import zipfile
import os

def unzip(path):
  zfile = zipfile.ZipFile(path)
  files = []
  for name in zfile.namelist():
    (dirname, filename) = os.path.split(name)
    if not (dirname.endswith('MODEL') or filename.endswith('.m') or filename == 'MANUAL.txt' or filename == 'README.txt'):
      files.append(name)
      if filename == '':
        if not os.path.exists(dirname):
          os.makedirs(dirname)
      else:
        fd = open(name, 'wb')
        fd.write(zfile.read(name))
        fd.close()
  zfile.close()
  return files

files = unzip('/content/ADL_Dataset.zip')

print('files unzipped !')
print(files)

In [0]:
import glob

directories = glob.glob("../content/HMP_Dataset/*")
#print(directories)

data = {}
x = np.empty((1,3))
i = 0
for filepath in files:
  if not filepath.endswith('.txt'):
    if np.size(x) > 3:
      data[i] = x[1:]
      i = i+1
    x = np.empty((1,3))
  else:
    fileData = np.loadtxt(filepath, delimiter=' ')
    x = np.append(x, fileData, axis=0)

#Last directory data was left behind 
#added it to the array of data
data[i] = x

print('Data loaded !', len(data))

In [0]:
vectored_data = {}
for i in range(14):
  x = np.empty((1,96))
  for j in range(0,np.size(data[i], axis=0)-32,32):
    x = np.append(x, data[i][j:j+32].reshape(1,96), axis=0)
  vectored_data[i] = x[1:]

In [0]:
count = 0
for i in range(14):
  count = count + len(data[i])
print(count)


In [0]:
from sklearn.cluster import KMeans

y_kmeans = {}
test = {}

for i in range(14):
  d = vectored_data[i]
  train_size = (len(d)*2)//3
  train = d[:train_size]
  test[i] = d[train_size:]
  kmeans = KMeans(n_clusters=40)
  kmeans.fit(train)
  y_kmeans[i] = kmeans.predict(train)

In [0]:
def histo(y_kmeans):
  histo = np.empty((14,40))
  for i in range(14):
    for j in range(40):
      idx = np.where(y_kmeans[i][:] == j)
      histo[i, j] = np.size(idx)
  return histo

train_histo = histo(y_kmeans)

In [0]:
test_kmeans = {}

for i in range(14):
  kmeans = KMeans(n_clusters=40)
  kmeans.fit(test[i])
  test_kmeans[i] = kmeans.predict(test[i])

test_histo = histo(test_kmeans)

In [0]:
from sklearn.metrics  import euclidean_distances

for i in range(14):
  for j in range(14):
    print(np.linalg.norm(train_histo[j]-test_histo[i]))
  print('\n')

'''
x = euclidean_distances(train_histo, test_histo, squared=True)
print(x[0,0])

print(numpy.linalg.norm(train_histo[i]-test_histo[j]))
''' 
